In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, cross_val_score
from sklearn.metrics import (
    precision_score,
    recall_score,
    f1_score,
    confusion_matrix,
    accuracy_score,
)
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

import optuna

import os

c:\Users\Dell-G5\anaconda3\envs\general\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
scenario_datasets = os.listdir("./Scenarios")
scenario_datasets = [f"./Scenarios/{x}" for x in scenario_datasets]
scenario_datasets

['./Scenarios/dataset_scenario_1_N.csv',
 './Scenarios/dataset_scenario_1_S.csv',
 './Scenarios/dataset_scenario_2_N.csv',
 './Scenarios/dataset_scenario_2_S.csv',
 './Scenarios/dataset_scenario_3_N.csv',
 './Scenarios/dataset_scenario_3_S.csv',
 './Scenarios/dataset_scenario_4_N.csv',
 './Scenarios/dataset_scenario_4_S.csv',
 './Scenarios/dataset_scenario_5_N.csv',
 './Scenarios/dataset_scenario_5_S.csv',
 './Scenarios/dataset_scenario_6_N.csv',
 './Scenarios/dataset_scenario_6_S.csv',
 './Scenarios/dataset_scenario_7_N.csv',
 './Scenarios/dataset_scenario_7_S.csv',
 './Scenarios/dataset_scenario_8_N.csv',
 './Scenarios/dataset_scenario_8_S.csv']

In [3]:
scenarios_description = [
    {"name": "Scenario 1_N", "encoding": "Label encoding", "remove_outliers": False, "remove_errors": False, "scaling": "MinMax"},
    {"name": "Scenario 1_S", "encoding": "Label encoding", "remove_outliers": False, "remove_errors": False, "scaling": "Standard"},
    {"name": "Scenario 2_N", "encoding": "Label encoding", "remove_outliers": False, "remove_errors": False,
     "impute_zeros": {"RestingBP": "mean", "Cholesterol": "mean"}, "oldpeak_abs": True, "scaling": "MinMax"},
    {"name": "Scenario 2_S", "encoding": "Label encoding", "remove_outliers": False, "remove_errors": False,
     "impute_zeros": {"RestingBP": "mean", "Cholesterol": "mean"}, "oldpeak_abs": True, "scaling": "Standard"},
    {"name": "Scenario 3_N", "encoding": "Label encoding", "remove_outliers": True, "remove_errors": True, "scaling": "None"},
    {"name": "Scenario 3_S", "encoding": "Label encoding", "remove_outliers": True, "remove_errors": True, "scaling": "Standard"},
    {"name": "Scenario 4_N", "encoding": "Label encoding", "remove_outliers": "replace_with_mean", "remove_errors": True, "scaling": "MinMax"},
    {"name": "Scenario 4_S", "encoding": "Label encoding", "remove_outliers": "replace_with_mean", "remove_errors": True, "scaling": "Standard"},
    {"name": "Scenario 5_N", "encoding": {"Sex": "Label", "ExerciseAngina": "Label", "ST_Slope": "Label", "RestingECG": "Label", "ChestPainType": "OneHot"},
     "remove_outliers": False, "remove_errors": False, "scaling": "MinMax"},
    {"name": "Scenario 5_S", "encoding": {"Sex": "Label", "ExerciseAngina": "Label", "ST_Slope": "Label", "RestingECG": "Label", "ChestPainType": "OneHot"},
     "remove_outliers": False, "remove_errors": False, "scaling": "Standard"},
    {"name": "Scenario 6_N", "encoding": {"Sex": "Label", "ExerciseAngina": "Label", "ST_Slope": "Label", "RestingECG": "Label", "ChestPainType": "OneHot"},
     "remove_outliers": False, "remove_errors": True, "scaling": "None"},
    {"name": "Scenario 6_S", "encoding": {"Sex": "Label", "ExerciseAngina": "Label", "ST_Slope": "Label", "RestingECG": "Label", "ChestPainType": "OneHot"},
     "remove_outliers": False, "remove_errors": True, "scaling": "Standard"},
    {"name": "Scenario 7_N", "encoding": {"Sex": "Label", "ExerciseAngina": "Label", "ST_Slope": "Label", "RestingECG": "Label", "ChestPainType": "OneHot"},
     "remove_outliers": True, "remove_errors": True, "scaling": "MinMax"},
    {"name": "Scenario 7_S", "encoding": {"Sex": "Label", "ExerciseAngina": "Label", "ST_Slope": "Label", "RestingECG": "Label", "ChestPainType": "OneHot"},
     "remove_outliers": True, "remove_errors": True, "scaling": "Standard"},
    {"name": "Scenario 8_N", "encoding": {"Sex": "Label", "ExerciseAngina": "Label", "ST_Slope": "Label", "RestingECG": "Label", "ChestPainType": "OneHot"},
     "remove_outliers": "replace_with_mean", "remove_errors": True, "scaling": "None"},
    {"name": "Scenario 8_S", "encoding": {"Sex": "Label", "ExerciseAngina": "Label", "ST_Slope": "Label", "RestingECG": "Label", "ChestPainType": "OneHot"},
     "remove_outliers": "replace_with_mean", "remove_errors": True, "scaling": "Standard"},
]


In [4]:
def get_search_hyperparameters(model_type):
    if model_type == "NaiveBayes":
        return {}
    elif model_type == "SVM":
        return {
            "C": [0.1, 1, 10, 100, 1000],
            "gamma": [1, 0.1, 0.01, 0.001, 0.0001],
            "kernel": ["rbf", "linear", "poly", "sigmoid"],
        }
    else:
        return {}

In [38]:
class optuna_tuner:
    def __init__(self, model, X_train, y_train, direction="maximize", n_trials=100, n_jobs=-1, scoring="accuracy", cv=5):
        self.model = model
        self.X_train = X_train
        self.y_train = y_train
        self.direction = direction
        self.n_trials = n_trials
        self.n_jobs = n_jobs
        self.scoring = scoring
        self.cv = cv

    def objective(self, trial):
        if isinstance(self.model, GaussianNB):
            pass
        elif isinstance(self.model, SVC):
            model = SVC(
                C=trial.suggest_float("C", 1e-3, 10),
                gamma=trial.suggest_float("gamma", 1e-3, 10),
                kernel=trial.suggest_categorical("kernel", ["rbf", "linear", "poly", "sigmoid"]),
            )
            scores = cross_val_score(model, self.X_train, self.y_train, cv=self.cv, scoring=self.scoring, n_jobs=self.n_jobs)
            return scores.mean()
        
    
    def optimize_study(self):
        study = optuna.create_study(direction=self.direction)
        study.optimize(self.objective, n_trials=self.n_trials, n_jobs=self.n_jobs)
        return study.best_params

    

In [31]:
class ModelTrainer:
    def __init__(self, dataset, target_column, model):
        self.dataset = dataset
        self.target_column = target_column
        self.X = dataset.drop(target_column, axis=1)
        self.y = dataset[target_column]
        self.model = model
        self.results = {}

    def split_data(self, test_size=0.2, val_size=0.1, random_state=42):
        X_train, X_temp, y_train, y_temp = train_test_split(
            self.X, self.y, test_size=test_size + val_size, random_state=random_state
        )
        val_ratio = val_size / (test_size + val_size)
        self.X_val, self.X_test, self.y_val, self.y_test = train_test_split(
            X_temp, y_temp, test_size=val_ratio, random_state=random_state
        )
        self.X_train, self.y_train = X_train, y_train

    def tune_and_train(self,search_type , search_config, param_grid):

        clf = self.model
        if search_type == "grid":
            search = GridSearchCV(clf, param_grid, cv=search_config["cv"], scoring=search_config["scoring"])
        elif search_type == "random":
            search = RandomizedSearchCV(clf, param_grid, cv=search_config["cv"], scoring=search_config["scoring"], n_iter=search_config["n_iter"])
        elif search_type == "optuna":
            search = optuna_tuner(self.model, self.X_train, self.y_train, search_config["direction"], search_config["n_trials"], search_config["n_jobs"], search_config["scoring"], search_config["cv"])

        best_params = None
        if isinstance(search, optuna_tuner):
            best_params = search.optimize_study()
            self.model = self.model.set_params(**best_params)
        elif search_type == "grid" or search_type == "random":
            search.fit(self.X_train, self.y_train)
            best_params = search.best_params_
            self.model = search.best_estimator_

        print("Best hyperparameters: ", best_params)

        # TODO: Get scores for train, val, and test sets


    def test_and_evaluate(self):
        pass

    def plot_results(self):
        pass


In [32]:
search_config_grid = {
    "cv": 5,
    "scoring": "accuracy",
}
search_config_random = {
    "cv": 5,
    "scoring": "accuracy",
    "n_iter": 50,
}
search_config_optuna = {
    "direction": "maximize",
    "n_trials": 100,
    "n_jobs": -1,
    "scoring": "accuracy",
    "cv": 5,
}

target_column = "HeartDisease"

In [33]:
def run_scenario(scenario, model_type, search_type, search_config, param_grid):
    dataset = pd.read_csv(scenario)
    model = None
    if model_type == "NaiveBayes":
        pass
    elif model_type == "SVM":
        model = SVC()

    trainer = ModelTrainer(dataset, target_column, model)
    trainer.split_data()
    trainer.tune_and_train(search_type, search_config, param_grid)
    # trainer.test_and_evaluate()
    # trainer.plot_results()

# Naive Bayes

# SVM

## 1. Grid Search

In [14]:
for scenario,scenario_description in zip(scenario_datasets, scenarios_description):
    print(f"Scenario Name: {scenario_description['name']}")
    print(f"  Encoding: {scenario_description['encoding']}")
    print(f"  Remove Outliers: {scenario_description['remove_outliers']}")
    print(f"  Remove Errors: {scenario_description.get('remove_errors', 'Not Specified')}")
    print(f"  Impute Zeros: {scenario_description.get('impute_zeros', 'None')}")
    print(f"  Oldpeak Absolute: {scenario_description.get('oldpeak_abs', 'Not Specified')}")
    print(f"  Scaling: {scenario_description['scaling']}")
    run_scenario(scenario, "SVM", "grid", search_config_grid, get_search_hyperparameters("SVM"))

    print("-" * 40)
    print("\n\n")

Scenario Name: Scenario 1_N
  Encoding: Label encoding
  Remove Outliers: False
  Remove Errors: False
  Impute Zeros: None
  Oldpeak Absolute: Not Specified
  Scaling: MinMax
Best hyperparameters:  {'C': 0.1, 'gamma': 1, 'kernel': 'poly'}
----------------------------------------



Scenario Name: Scenario 1_S
  Encoding: Label encoding
  Remove Outliers: False
  Remove Errors: False
  Impute Zeros: None
  Oldpeak Absolute: Not Specified
  Scaling: Standard
Best hyperparameters:  {'C': 1, 'gamma': 0.1, 'kernel': 'rbf'}
----------------------------------------



Scenario Name: Scenario 2_N
  Encoding: Label encoding
  Remove Outliers: False
  Remove Errors: False
  Impute Zeros: {'RestingBP': 'mean', 'Cholesterol': 'mean'}
  Oldpeak Absolute: True
  Scaling: MinMax
Best hyperparameters:  {'C': 1000, 'gamma': 0.001, 'kernel': 'rbf'}
----------------------------------------



Scenario Name: Scenario 2_S
  Encoding: Label encoding
  Remove Outliers: False
  Remove Errors: False
  Impute 

## 2. Random Search

In [15]:
for scenario,scenario_description in zip(scenario_datasets, scenarios_description):
    print(f"Scenario Name: {scenario_description['name']}")
    print(f"  Encoding: {scenario_description['encoding']}")
    print(f"  Remove Outliers: {scenario_description['remove_outliers']}")
    print(f"  Remove Errors: {scenario_description.get('remove_errors', 'Not Specified')}")
    print(f"  Impute Zeros: {scenario_description.get('impute_zeros', 'None')}")
    print(f"  Oldpeak Absolute: {scenario_description.get('oldpeak_abs', 'Not Specified')}")
    print(f"  Scaling: {scenario_description['scaling']}")
    run_scenario(scenario, "SVM", "random", search_config_random, get_search_hyperparameters("SVM"))

    print("-" * 40)
    print("\n\n")

Scenario Name: Scenario 1_N
  Encoding: Label encoding
  Remove Outliers: False
  Remove Errors: False
  Impute Zeros: None
  Oldpeak Absolute: Not Specified
  Scaling: MinMax
Best hyperparameters:  {'kernel': 'poly', 'gamma': 1, 'C': 0.1}
----------------------------------------



Scenario Name: Scenario 1_S
  Encoding: Label encoding
  Remove Outliers: False
  Remove Errors: False
  Impute Zeros: None
  Oldpeak Absolute: Not Specified
  Scaling: Standard
Best hyperparameters:  {'kernel': 'rbf', 'gamma': 0.1, 'C': 1}
----------------------------------------



Scenario Name: Scenario 2_N
  Encoding: Label encoding
  Remove Outliers: False
  Remove Errors: False
  Impute Zeros: {'RestingBP': 'mean', 'Cholesterol': 'mean'}
  Oldpeak Absolute: True
  Scaling: MinMax
Best hyperparameters:  {'kernel': 'rbf', 'gamma': 0.001, 'C': 1000}
----------------------------------------



Scenario Name: Scenario 2_S
  Encoding: Label encoding
  Remove Outliers: False
  Remove Errors: False
  Impute 

## 3. Bayesian Optimization (Optuna)

In [39]:
for scenario,scenario_description in zip(scenario_datasets, scenarios_description):
    print(f"Scenario Name: {scenario_description['name']}")
    print(f"  Encoding: {scenario_description['encoding']}")
    print(f"  Remove Outliers: {scenario_description['remove_outliers']}")
    print(f"  Remove Errors: {scenario_description.get('remove_errors', 'Not Specified')}")
    print(f"  Impute Zeros: {scenario_description.get('impute_zeros', 'None')}")
    print(f"  Oldpeak Absolute: {scenario_description.get('oldpeak_abs', 'Not Specified')}")
    print(f"  Scaling: {scenario_description['scaling']}")
    # here the get_search_hyperparameters("SVM") is not used as the optuna_tuner class is used to tune the hyperparameters
    run_scenario(scenario, "SVM", "optuna", search_config_optuna, get_search_hyperparameters("SVM"))

    print("-" * 40)
    print("\n\n")

[I 2024-11-30 12:48:38,640] A new study created in memory with name: no-name-01d637bc-5e07-422f-b5f8-53978850d05e
[I 2024-11-30 12:48:38,748] Trial 0 finished with value: 0.8473231589147285 and parameters: {'C': 7.26237994975051, 'gamma': 0.2937391886759249, 'kernel': 'rbf'}. Best is trial 0 with value: 0.8473231589147285.
[I 2024-11-30 12:48:38,753] Trial 2 finished with value: 0.8410610465116278 and parameters: {'C': 2.9209376292464078, 'gamma': 0.9646524570987972, 'kernel': 'rbf'}. Best is trial 0 with value: 0.8473231589147285.
[I 2024-11-30 12:48:38,788] Trial 1 finished with value: 0.8442223837209302 and parameters: {'C': 8.343660802130607, 'gamma': 8.606156154741576, 'kernel': 'linear'}. Best is trial 0 with value: 0.8473231589147285.
[I 2024-11-30 12:48:38,810] Trial 5 finished with value: 0.8442223837209302 and parameters: {'C': 8.980044584801023, 'gamma': 3.1166618536542146, 'kernel': 'linear'}. Best is trial 0 with value: 0.8473231589147285.


Scenario Name: Scenario 1_N
  Encoding: Label encoding
  Remove Outliers: False
  Remove Errors: False
  Impute Zeros: None
  Oldpeak Absolute: Not Specified
  Scaling: MinMax


[I 2024-11-30 12:48:38,846] Trial 3 finished with value: 0.8348594961240309 and parameters: {'C': 2.1330174327289186, 'gamma': 0.030188244511243783, 'kernel': 'rbf'}. Best is trial 0 with value: 0.8473231589147285.
[I 2024-11-30 12:48:38,884] Trial 6 finished with value: 0.8457848837209301 and parameters: {'C': 3.7677922170115536, 'gamma': 9.307317678732998, 'kernel': 'linear'}. Best is trial 0 with value: 0.8473231589147285.
[I 2024-11-30 12:48:38,935] Trial 11 finished with value: 0.8457848837209301 and parameters: {'C': 5.824631888460823, 'gamma': 2.170918268018874, 'kernel': 'linear'}. Best is trial 0 with value: 0.8473231589147285.
[I 2024-11-30 12:48:39,000] Trial 8 finished with value: 0.8052446705426357 and parameters: {'C': 9.284862209819115, 'gamma': 6.454072467050267, 'kernel': 'rbf'}. Best is trial 0 with value: 0.8473231589147285.
[I 2024-11-30 12:48:39,094] Trial 9 finished with value: 0.8083212209302324 and parameters: {'C': 7.348239499174422, 'gamma': 9.033334848494034,

Best hyperparameters:  {'C': 7.26237994975051, 'gamma': 0.2937391886759249, 'kernel': 'rbf'}
----------------------------------------



Scenario Name: Scenario 1_S
  Encoding: Label encoding
  Remove Outliers: False
  Remove Errors: False
  Impute Zeros: None
  Oldpeak Absolute: Not Specified
  Scaling: Standard


[I 2024-11-30 12:48:43,459] Trial 1 finished with value: 0.7741157945736434 and parameters: {'C': 5.623265042952017, 'gamma': 3.9504735579550343, 'kernel': 'poly'}. Best is trial 2 with value: 0.8442223837209302.
[I 2024-11-30 12:48:43,463] Trial 7 finished with value: 0.7741157945736434 and parameters: {'C': 7.273910000216464, 'gamma': 2.8857063810496086, 'kernel': 'poly'}. Best is trial 2 with value: 0.8442223837209302.
[I 2024-11-30 12:48:43,487] Trial 8 finished with value: 0.6541182170542635 and parameters: {'C': 1.4147479172388433, 'gamma': 3.694055383501027, 'kernel': 'rbf'}. Best is trial 2 with value: 0.8442223837209302.
[I 2024-11-30 12:48:43,491] Trial 5 finished with value: 0.6012112403100776 and parameters: {'C': 2.1962841415794334, 'gamma': 7.508569191399667, 'kernel': 'rbf'}. Best is trial 2 with value: 0.8442223837209302.
[I 2024-11-30 12:48:43,542] Trial 4 finished with value: 0.7741157945736434 and parameters: {'C': 8.205324561328393, 'gamma': 9.87159462457618, 'kerne

...

Best hyperparameters:  {'C': 0.4732614657967059, 'gamma': 0.039675789414968216, 'kernel': 'linear'}
----------------------------------------



Scenario Name: Scenario 8_N
  Encoding: {'Sex': 'Label', 'ExerciseAngina': 'Label', 'ST_Slope': 'Label', 'RestingECG': 'Label', 'ChestPainType': 'OneHot'}
  Remove Outliers: replace_with_mean
  Remove Errors: True
  Impute Zeros: None
  Oldpeak Absolute: Not Specified
  Scaling: None


[I 2024-11-30 12:49:22,547] Trial 7 finished with value: 0.6509932170542635 and parameters: {'C': 6.204415268338885, 'gamma': 1.000711122659414, 'kernel': 'sigmoid'}. Best is trial 5 with value: 0.8410852713178294.
[I 2024-11-30 12:49:22,589] Trial 11 finished with value: 0.8410852713178294 and parameters: {'C': 7.169253149988003, 'gamma': 3.67812384048594, 'kernel': 'linear'}. Best is trial 5 with value: 0.8410852713178294.
[I 2024-11-30 12:49:22,650] Trial 9 finished with value: 0.8395227713178294 and parameters: {'C': 4.331988315663916, 'gamma': 5.010618671210604, 'kernel': 'linear'}. Best is trial 5 with value: 0.8410852713178294.
[I 2024-11-30 12:49:22,866] Trial 10 finished with value: 0.6028706395348837 and parameters: {'C': 9.80973014579516, 'gamma': 2.5747553375732632, 'kernel': 'sigmoid'}. Best is trial 5 with value: 0.8410852713178294.
[I 2024-11-30 12:49:22,909] Trial 12 finished with value: 0.8410852713178294 and parameters: {'C': 8.782227282635997, 'gamma': 4.953738680490

Best hyperparameters:  {'C': 1.1758009960696496, 'gamma': 8.599120983741958, 'kernel': 'linear'}
----------------------------------------



Scenario Name: Scenario 8_S
  Encoding: {'Sex': 'Label', 'ExerciseAngina': 'Label', 'ST_Slope': 'Label', 'RestingECG': 'Label', 'ChestPainType': 'OneHot'}
  Remove Outliers: replace_with_mean
  Remove Errors: True
  Impute Zeros: None
  Oldpeak Absolute: Not Specified
  Scaling: Standard


[I 2024-11-30 12:49:26,901] Trial 9 finished with value: 0.7025193798449612 and parameters: {'C': 3.79926253353986, 'gamma': 4.4336824999266256, 'kernel': 'sigmoid'}. Best is trial 2 with value: 0.7819282945736434.
[I 2024-11-30 12:49:26,914] Trial 7 finished with value: 0.7446220930232558 and parameters: {'C': 6.527718345997648, 'gamma': 7.2511488955233, 'kernel': 'sigmoid'}. Best is trial 2 with value: 0.7819282945736434.
[I 2024-11-30 12:49:26,922] Trial 1 finished with value: 0.5623183139534884 and parameters: {'C': 9.506901737907056, 'gamma': 8.190319485803665, 'kernel': 'rbf'}. Best is trial 2 with value: 0.7819282945736434.
[I 2024-11-30 12:49:26,932] Trial 6 finished with value: 0.7506904069767442 and parameters: {'C': 0.13798686644552763, 'gamma': 6.904526119543761, 'kernel': 'poly'}. Best is trial 2 with value: 0.7819282945736434.
[I 2024-11-30 12:49:26,938] Trial 8 finished with value: 0.8410852713178294 and parameters: {'C': 6.0766852180914865, 'gamma': 8.295732736461876, '

Best hyperparameters:  {'C': 6.0766852180914865, 'gamma': 8.295732736461876, 'kernel': 'linear'}
----------------------------------------



